In [1]:
import networkx as nx
import numpy as np
import random
import glob
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import colors
from pyadigraph import Adigraph

In [2]:
class RandomWeightedGraph(nx.Graph):
    def __init__(self, nodes:int):
        super().__init__(nx.bipartite.random_graph(nodes, nodes, 0.5).edges())
        for (u, v) in self.edges():
            self.edges[u,v]['weight'] = np.random.randint(10, 100)
            
    def weight(self, edge):
        return self.edges[edge]['weight']

In [3]:
class MinimumSpanningTree(list):
    def __init__(self, G: nx.Graph, root: int, describe: bool = False):
        self._original_G, self._G, self._root = G, G, root
        self.counter = 0
        files = glob.glob('../images/kruskal/*')
        for f in files:
            os.remove(f)
        if describe:
            self._setup_describe()

    def _setup_describe(self):
        self._pos = nx.spring_layout(self._G, iterations=10000)
        gray, self._dark_gray = '#eeeeee', '#555555'
        self._node_colors = [gray for n in self._G]
        self._edge_colors = [gray for e in self._G.edges]
        positions = np.array(list(self._pos.values()))
        self._min_x, self._min_y = np.min(positions, axis=0) - 0.1
        self._max_x, self._max_y = np.max(positions, axis=0) + 0.1

    def _plot_graphs(self, graphs: list, title: str):
        a = Adigraph(
            layout=self._pos,
            weights=nx.get_edge_attributes(self._original_G, 'weight'),
            style="-",
            row_size=4,
            caption=title,
            directed=False,
            edges_color_fallback="gray!80",
            vertices_color_fallback="gray")
        
        comulative_colors = {}
        comulative_edges_colors = {}
        comulative_edges_width = {}
        comulative_vertices_width = {}

        for i, ((G, title), color) in enumerate(
                zip(graphs, ['red!90', 'cyan!90', 'magenta!90', 'pink!90', 'orange!90', 'blue!90', 'green!90', 'Melon!90'])):
            colors = {v: color for v in G.nodes}
            edges_color = {e: color for e in G.edges}
            edges_width = {e: 1 for e in G.edges}
            vertices_width = {v: 1 for v in G.nodes}
            comulative_colors.update(colors)
            comulative_edges_colors.update(edges_color)
            comulative_edges_width.update(edges_width)
            comulative_vertices_width.update(vertices_width)
            a.add_graph(
                self._original_G,
                caption=title,
                vertices_color=colors,
                edges_color=edges_color,
                edges_width=edges_width,
                vertices_width=vertices_width)

        a.add_graph(
                self._original_G,
                caption=title,
                vertices_color=comulative_colors,
                edges_color=comulative_edges_colors,
                edges_width=comulative_edges_width,
                vertices_width=comulative_vertices_width)


        a.save("../chapters/kruskal/{c}.tex".format(c=self.counter))
        self.counter += 1

    def _describe_start(self, root: int):
        tree = nx.Graph()
        tree.add_node(root)
        colors = ['r' for n in self._original_G]
        self._plot_graphs([(self._original_G, colors, "Initial Graph"),
                           (tree, colors, "Initial Tree")],
                          "Initial Conditions")

In [4]:
class KruskalTree(MinimumSpanningTree):
    def __init__(self, G: nx.Graph, root: int, describe: bool = False):
        super().__init__(G, root, describe=describe)
        [self.append([(v, v)]) for v in G]
    
    def run(self):
        weights = nx.get_edge_attributes(G, 'weight')
        self._edges = sorted(weights, key=weights.get, reverse=True)
        while not self._is_spanning():
            self.iteration()
    
    def iteration(self):
        edge = u, v = self._edges.pop()
        set_u = self._get_node_component(u)
        set_v = self._get_node_component(v)
        if set_u != set_v:
            self[set_u] += self[set_v] + [(u, v)]
            del self[set_v]
        return set_u, set_v, u, v
        

    def _get_node_component(self, node: int):
        for i, component in enumerate(self):
            for edge in component:
                if node in edge:
                    return i
        return None

    def _is_spanning(self):
        return len(self) == 1


In [5]:
class GraphicalKruskalTree(KruskalTree):
    def __init__(self, G: nx.Graph, root: int):
        super().__init__(G, root, True)

    def run(self):
        self._plot_graphs(
            [(nx.Graph(c), "".format(c=i)) for i,c in enumerate(self)],
            "Initial conditions")
        self._i=0
        super().run()

    def iteration(self):
        set_u, set_v, u, v = super().iteration()
        self._plot_graphs(
            [(nx.Graph(c), "".format(j=j))
             for j, c in enumerate(self)],
            "Iteration {i}: Merging component {u} with component {v} via edge ({a}, {b})"
            .format(i=self._i, u=set_u, v=set_v, a=u, b=v))
        self._i+=1
    
    def _create_iteration_graph(self, costs: dict, neighbors: list,
                                node: int) -> tuple:
        iteration = nx.Graph()
        [iteration.add_node(v) for v in [*costs, node, *neighbors]]
        colors = [
            'g' if v in neighbors else 'm' if v == node else 'c'
            for v in iteration
        ]
        return iteration, colors

In [6]:
seed = 236
random.seed(seed)
np.random.seed(seed)
G = RandomWeightedGraph(3)
root = 1
end = 4

In [7]:
GraphicalKruskalTree(G, root).run()